In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model

### Register data

In [2]:
from federatedscope.contrib.data.mnist_data import call_my_data

register_data("mydata", call_my_data)

### Register model

In [3]:
from federatedscope.contrib.model.mnist_model import call_my_net
register_model("mynet2", call_my_net)

### Register trainer

In [4]:
from federatedscope.contrib.trainer.laplacian_trainer_no_graph import call_laplacian_trainer

register_trainer('laplacian_trainer_no_graph', call_laplacian_trainer)

### Register metric

In [5]:
from federatedscope.contrib.metrics.mnist_metric import call_my_metric

register_metric("mymetric", call_my_metric)

### Create the config

#### Set data, model, trainer and metric

In [6]:
from federatedscope.core.configs.config import global_cfg, CN

cfg = global_cfg.clone()
cfg.data.type = 'mydata'
cfg.data.root = 'data'
cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.1307], 'std': [0.3081]}]]
cfg.model.type = 'mynet2'
cfg.model.in_channels = 1
cfg.model.out_channels = 10
cfg.trainer.type = 'laplacian_trainer'
cfg.eval.metric = ['mymetric']

#### configure other options

In [7]:
cfg.use_gpu = True
cfg.best_res_update_round_wise_key = "test_loss"

cfg.federate.mode = 'standalone'
cfg.federate.local_update_steps = 1
cfg.federate.total_round_num = 2
cfg.federate.sample_client_num = 50
cfg.federate.client_num = 50

cfg.train.optimizer.lr = 0.1
cfg.train.optimizer.weight_decay = 0.0
cfg.grad.grad_clip = 5.0
cfg.train.local_update_steps = 5
cfg.train.batch_or_epoch = 'epoch'
cfg.criterion.type = 'CrossEntropyLoss'
cfg.seed = 123
cfg.eval.best_res_update_round_wise_key = "test_loss"
cfg.params = CN()
cfg.params.alpha=0.01
cfg.params.csd_importance=1
cfg.params.eps=1e-5
cfg.params.p=0.
cfg.eval.metrics = ['acc']

### Start the FL prosess

In [8]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2022-09-06 20:30:59,688 (register:12)WARNING: Key mydata is already pre-defined, overwritten.


In [9]:
from federatedscope.contrib.workers.laplacian_client_no_graph import LaplacianClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

In [10]:

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2022-09-06 20:30:59,715 (utils:129)INFO: the current machine is at 127.0.1.1
2022-09-06 20:30:59,716 (utils:131)INFO: the current dir is /home/michael/Master thesis/Code/FederatedScope_thesis
2022-09-06 20:30:59,716 (utils:132)INFO: the output dir is exp/FedAvg_mynet2_on_mydata_lr0.1_lstep5_/sub_exp_20220906203059


In [11]:
from federatedscope.contrib.data.mnist_data import call_my_data

register_data("mydata", call_my_data)

2022-09-06 20:30:59,727 (register:12)WARNING: Key mydata is already pre-defined, overwritten.


In [12]:

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2022-09-06 20:30:59,741 (utils:129)INFO: the current machine is at 127.0.1.1
2022-09-06 20:30:59,742 (utils:131)INFO: the current dir is /home/michael/Master thesis/Code/FederatedScope_thesis
2022-09-06 20:30:59,743 (utils:132)INFO: the output dir is exp/FedAvg_mynet2_on_mydata_lr0.1_lstep5_/sub_exp_20220906203059/FedAvg_mynet2_on_mydata_lr0.1_lstep5__


train per client: 20


In [13]:
Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianClient,
                       config=cfg.clone())
Fed_runner.run()

2022-09-06 20:31:03,488 (fed_runner:249)INFO: Server #0 has been set up ... 
2022-09-06 20:31:03,581 (trainer:50)WARNING: Will not use monitor in trainer with class <class 'federatedscope.contrib.trainer.laplacian_trainer_no_graph.LaplacianTrainerNoGraph'>
2022-09-06 20:31:03,582 (fed_runner:302)INFO: Client 1 has been set up ... 
2022-09-06 20:31:03,677 (trainer:50)WARNING: Will not use monitor in trainer with class <class 'federatedscope.contrib.trainer.laplacian_trainer_no_graph.LaplacianTrainerNoGraph'>
2022-09-06 20:31:03,679 (fed_runner:302)INFO: Client 2 has been set up ... 
2022-09-06 20:31:03,766 (trainer:50)WARNING: Will not use monitor in trainer with class <class 'federatedscope.contrib.trainer.laplacian_trainer_no_graph.LaplacianTrainerNoGraph'>
2022-09-06 20:31:03,767 (fed_runner:302)INFO: Client 3 has been set up ... 
2022-09-06 20:31:03,865 (trainer:50)WARNING: Will not use monitor in trainer with class <class 'federatedscope.contrib.trainer.laplacian_trainer_no_graph.L

{'client_best_individual': {'test_loss': 258.9394359588623,
  'test_avg_loss': 1.2946971797943114,
  'test_acc': 0.6,
  'test_total': 200.0},
 'client_summarized_weighted_avg': {'test_loss': 303.09110340118406,
  'test_avg_loss': 1.5154555170059205,
  'test_acc': 0.6951,
  'test_total': 200.0},
 'client_summarized_avg': {'test_loss': 303.09110340118406,
  'test_avg_loss': 1.5154555170059203,
  'test_acc': 0.6950999999999999,
  'test_total': 200.0},
 'client_summarized_fairness': {'test_loss_top_decile': 325.0300188064575,
  'test_loss_bottom_decile': 281.0638256072998,
  'test_loss_std': 19.221526965696587,
  'test_total': 200.0,
  'test_avg_loss_std': 0.0961076348284829,
  'test_avg_loss_bottom_decile': 1.405319128036499,
  'test_avg_loss_top_decile': 1.6251500940322876,
  'test_acc_std': 0.052311471017359085,
  'test_acc_bottom_decile': 0.63,
  'test_acc_top_decile': 0.76}}

In [14]:
data

{1: {'train': <torch.utils.data.dataloader.DataLoader at 0x7f1192df18b0>,
  'test': <torch.utils.data.dataloader.DataLoader at 0x7f1192df16a0>},
 2: {'train': <torch.utils.data.dataloader.DataLoader at 0x7f1192dee760>,
  'test': <torch.utils.data.dataloader.DataLoader at 0x7f1192dee670>},
 3: {'train': <torch.utils.data.dataloader.DataLoader at 0x7f1192dee5e0>,
  'test': <torch.utils.data.dataloader.DataLoader at 0x7f1192dee2b0>},
 4: {'train': <torch.utils.data.dataloader.DataLoader at 0x7f1192dee160>,
  'test': <torch.utils.data.dataloader.DataLoader at 0x7f1192dee310>},
 5: {'train': <torch.utils.data.dataloader.DataLoader at 0x7f1192decee0>,
  'test': <torch.utils.data.dataloader.DataLoader at 0x7f1192dec970>},
 6: {'train': <torch.utils.data.dataloader.DataLoader at 0x7f1192dec820>,
  'test': <torch.utils.data.dataloader.DataLoader at 0x7f1192dec6a0>},
 7: {'train': <torch.utils.data.dataloader.DataLoader at 0x7f1192dec460>,
  'test': <torch.utils.data.dataloader.DataLoader at 0x7

In [15]:
cfg

CN({'backend': 'torch', 'use_gpu': True, 'verbose': 1, 'print_decimal_digits': 6, 'device': -1, 'seed': 123, 'cfg_file': '', 'outdir': 'exp/FedAvg_mynet2_on_mydata_lr0.1_lstep5_/sub_exp_20220906195741/FedAvg_mynet2_on_mydata_lr0.1_lstep5__', 'expname': 'FedAvg_mynet2_on_mydata_lr0.1_lstep5__', 'expname_tag': '', 'hpo': CN({'working_folder': 'hpo', 'ss': '', 'num_workers': 0, 'init_cand_num': 16, 'log_scale': False, 'larger_better': False, 'scheduler': 'rs', 'plot_interval': 1, 'metric': 'client_summarized_weighted_avg.val_loss', 'sha': CN({'elim_round_num': 3, 'elim_rate': 3, 'budgets': []}), 'pbt': CN({'max_stage': 5, 'perf_threshold': 0.1}), 'fedex': CN({'use': False, 'ss': '', 'flatten_ss': True, 'eta0': -1.0, 'sched': 'auto', 'cutoff': 0.0, 'gamma': 0.0, 'num_arms': 16, 'diff': False}), 'table': CN({'ss': '', 'eps': 0.1, 'num': 27, 'idx': 0})}), 'data': CN({'root': 'data', 'type': 'mydata', 'args': [], 'splitter': '', 'splitter_args': [], 'transform': [['ToTensor'], ['Normalize', {

In [ ]:
data

In [ ]:
from federatedscope.contrib.model.mnist_model import MyNet

model = MyNet(0)